In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.extend([
    'openml-python-OpenMLActiveClassificationTask/',
    'openml-scikit-activeml/',
    '../../openml-python-OpenMLActiveClassificationTask/',
    '../../openml-scikit-activeml/',
])

from openml import config, tasks

config.start_using_configuration_for_example()

c:\Users\tpham\Desktop\Arbeit\openml-skactiveml\openml-python-OpenMLActiveClassificationTask\openml\config.py:184: UserWarning: Switching to the test server https://test.openml.org/api/v1/xml to not upload results to the live server. Using the test server may result in reduced performance of the API!
  warnings.warn(


In [2]:
task_id_clf = 119
task_clf = tasks.get_task(task_id_clf)

task_al = tasks.OpenMLActiveClassificationTask(
    task_type_id=tasks.TaskType.ACTIVE_CLASSIFICATION,
    task_type="ACTIVE_CLASSIFICATION",
    data_set_id=task_clf.dataset_id,
    target_name=task_clf.target_name,
    class_labels=task_clf.class_labels,
    evaluation_measure="predictive_accuracy",
    estimation_procedure_id=30,
    batch_size=1,
)

task_al.publish()

OpenML Active Classification Task
Task Type Description: https://test.openml.org/tt/TaskType.ACTIVE_CLASSIFICATION
Task ID..............: 1561
Task URL.............: https://test.openml.org/t/1561
Estimation Procedure.: None
Evaluation Measure...: predictive_accuracy
Target Feature.......: class
# of Classes.........: 2
Cost Matrix..........: Available

In [3]:
# Hot fix if datasplit generation is broken for AL tasks.
# from shutil import copyfile
# task = tasks.get_task(task_al.id, download_data=False)
# copyfile(
#     f"{config.cache_directory}/org/openml/test/tasks/{task_clf.id}/datasplits.arff",
#     f"{config.cache_directory}/org/openml/test/tasks/{task_id.al}/datasplits.arff"
# )
# copyfile(
#     f"{config.cache_directory}/org/openml/test/tasks/{task_clf.id}/datasplits.pkl.py3",
#     f"{config.cache_directory}/org/openml/test/tasks/{task_al.id}/datasplits.pkl.py3"
# )

c:\Users\tpham\Desktop\Arbeit\openml-skactiveml\openml-python-OpenMLActiveClassificationTask\openml\config.py:184: UserWarning: Switching to the test server https://test.openml.org/api/v1/xml to not upload results to the live server. Using the test server may result in reduced performance of the API!
  warnings.warn(
